In [7]:
import torch

def median_filter(image, kernel_size):
    """
    Применяет медианный фильтр к изображению с заданным размером ядра.
    
    Args:
        image: Тензор изображения в формате (C, H, W).
        kernel_size: Размер ядра.
    
    Returns:
        Отфильтрованный тензор того же формата.
    """
    pad = kernel_size // 2
    # Добавляем отраженный паддинг
    image_padded = torch.nn.functional.pad(
        image.unsqueeze(0), 
        [pad, pad, pad, pad], 
        mode='reflect'
    ).squeeze(0)
    
    # Создаем патчи с помощью unfold
    patches = image_padded.unfold(1, kernel_size, 1).unfold(2, kernel_size, 1)
    # patches.shape: (C, H, W, kernel_size, kernel_size)
    
    # Сортируем значения в патчах
    sorted_patches, _ = patches.reshape(*patches.shape[:3], -1).sort(dim=-1)
    
    # Вычисляем медиану (для четного количества берем нижнюю медиану)
    median_vals = sorted_patches[..., sorted_patches.shape[-1] // 2]
    
    return median_vals

# Пример использования
if __name__ == "__main__":
    # Создаем тестовое изображение 8x8 с градиентом
    image = torch.arange(64).float().view(1, 8, 8)
    print("Исходное изображение:")
    print(image.squeeze().int())
    
    for kernel_size in [3, 5, 10]:
        filtered = median_filter(image, kernel_size)
        print(f"\nМедианный фильтр с ядром {kernel_size}:")
        print(filtered.squeeze().int())

Исходное изображение:
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62, 63]], dtype=torch.int32)

Медианный фильтр с ядром 3:
tensor([[ 8,  8,  9, 10, 11, 12, 13, 14],
        [ 9,  9, 10, 11, 12, 13, 14, 14],
        [17, 17, 18, 19, 20, 21, 22, 22],
        [25, 25, 26, 27, 28, 29, 30, 30],
        [33, 33, 34, 35, 36, 37, 38, 38],
        [41, 41, 42, 43, 44, 45, 46, 46],
        [49, 49, 50, 51, 52, 53, 54, 54],
        [49, 50, 51, 52, 53, 54, 55, 55]], dtype=torch.int32)

Медианный фильтр с ядром 5:
tensor([[10, 10, 11, 12, 13, 14, 14, 14],
        [10, 10, 11, 12, 13, 14, 14, 14],
        [17, 17, 18, 19, 20, 21, 22, 22],
        [25, 25, 26, 27, 28, 29, 30, 30],
        [33, 33, 34, 35, 36, 37, 38, 38]